In [0]:
from pyspark.sql import Row
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.types import FloatType
from pyspark.sql.types import DoubleType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf
from pyspark.sql import functions as F
from pyspark.sql.functions import explode, col, udf, mean as _mean, stddev as _stddev, log, log10
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.functions import lit

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import NaiveBayes
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import MinMaxScaler
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [0]:
df = spark.read.orc('/savetable/train_select.orc')
dfTrain = df.withColumn("pha", col('pha').cast('Integer'))
dfTrain2 = dfTrain.select('pha', 'pseudo_target')

In [0]:
predictions = dfTrain2.withColumnRenamed("pha","label")
predictions = predictions.withColumnRenamed("pseudo_target","prediction")
predictions = predictions.withColumn("label", col('label').cast('Double'))
predictions = predictions.withColumn("prediction", col('prediction').cast('Double'))

In [0]:
predictionAndLabels = predictions.select("prediction", "label").rdd
metrics = BinaryClassificationMetrics(predictionAndLabels)
metricsMC = MulticlassMetrics(predictionAndLabels)

In [0]:
print("F1 score for pha = no :", metricsMC.fMeasure(label=0.0))
print("F1 score for pha = yes :", metricsMC.fMeasure(label=1.0))
print("Area under ROC =", metrics.areaUnderPR)

F1 score for pha = no : 0.9999659647621838
F1 score for pha = yes : 0.9846246608381067
Area under ROC = 0.9754466062147268

In [0]:
dfTest = spark.read.orc('/savetable/test_clean.orc')
dfTest = dfTest.withColumn("pha", col('pha').cast('Integer'))
dfTest2 = dfTest.select('pha', 'pseudo_target')

In [0]:
predictions = dfTest.withColumnRenamed("pha","label")
predictions = predictions.withColumnRenamed("pseudo_target","prediction")
predictions = predictions.withColumn("label", col('label').cast('Double'))
predictions = predictions.withColumn("prediction", col('prediction').cast('Double'))

In [0]:
predictionAndLabels = predictions.select("prediction", "label").rdd
metrics = BinaryClassificationMetrics(predictionAndLabels)
metricsMC = MulticlassMetrics(predictionAndLabels)

In [0]:
print("F1 score for pha = no :", metricsMC.fMeasure(label=0.0))
print("F1 score for pha = yes :", metricsMC.fMeasure(label=1.0))
print("Area under ROC =", metrics.areaUnderPR)

F1 score for pha = no : 0.999970636676926
F1 score for pha = yes : 0.9866666666666666
Area under ROC = 0.9807042907147706